In [1]:
import matplotlib.pyplot as plt
from src import logger
from src.utils.common import read_yaml, log_error
from dataclasses import dataclass
from src.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
import cv2
import numpy as np
import os


In [2]:
os.chdir("../")

In [24]:
@dataclass
class ImagePreprocessingConfig:
    raw_path: str
    interim_path: str

class ConfigurationManager:
    def __init__(self):
        self.config=read_yaml(CONFIG_FILE_PATH)
        self.params=read_yaml(PARAMS_FILE_PATH)
        print(self.config)
    def get_image_preprocessing_config(self):
        params=ImagePreprocessingConfig(
            raw_path=self.config.data.extract_to,
            interim_path=self.config.data.interim_path
        )
        return params


class ImagePreprocessing:
    def __init__(self, config):
        self.config=config

    def process_image(self, image):
        corrected_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        corrected_image = corrected_image.astype(np.uint8)

        clahe = cv2.createCLAHE(clipLimit=70.0, tileGridSize=(8, 8))
        enhanced_image = clahe.apply(corrected_image)
        
        # Apply erosion to thin and darken lines
        kernel = np.ones((1, 1), np.uint8)
        thinned_image = cv2.erode(enhanced_image, kernel, iterations=200)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(5,5))
        enhanced_contrast = clahe.apply(thinned_image)

        smoothed = cv2.GaussianBlur(enhanced_contrast, (3, 3), 23)
        # Step 3: Sharpen the image
        sharpened = cv2.addWeighted(enhanced_contrast,2, smoothed, -0.5, 0)

        denoised_image = cv2.fastNlMeansDenoising(sharpened, None, h=20, 
                                                templateWindowSize=5, 
                                                searchWindowSize=25)
        kernel = np.ones((1, 1), np.uint8)
        erode_image = cv2.erode(denoised_image, kernel, iterations=200)
        return erode_image
    

    def read_image(self, path):
        image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        image = cv2.cvtColor(image).astype(np.uint8)
        return image
    
    @log_error(sucess_message=None, faliure_message="Location does not exists")
    def get_images_path(self):
        folder_dir = list(os.listdir(self.config.interim_path))
        for folder in folder_dir:
            if os.path.isdir(os.path.join(self.config.interim_path, folder)) != True:
                folder_dir.remove(folder)
        
        return folder_dir




In [25]:
if __name__ == "__main__":
    config_manager = ConfigurationManager()
    config_params = config_manager.get_image_preprocessing_config()

    ob = ImagePreprocessing(config_params)
    images_path = ob.get_images_path()

2025-01-13 18:47:22,845 - common - INFO - Yaml read successfully from config/config.yaml
2025-01-13 18:47:22,847 - common - INFO - Yaml read successfully from params.yaml
2025-01-13 18:47:22,848 - common - ERROR - Exception occured while reading yaml file from                         location: params.yaml
 First argument must be mapping or iterable


{'data': {'google_drive_url': 'https://drive.google.com/file/d/10ApjOcTo6tjO34q5et-ij-tbrSm0Waqc/view?usp=drive_link', 'download_path': 'data/external/data.zip', 'extract_to': 'data/raw', 'interim_path': 'data/interim', 'processed_path': 'data/processed'}}


In [27]:
images_path

['AB+', 'AB-', 'A+', 'B-', 'B+', 'A-', 'O-', 'O+']